In [ ]:
pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 3.8MB 47.2MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 512kB 54.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=edcceaa27d21168102618e78eab48d7960a318eea14d694b3dff0b75641cc178
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

True

In [ ]:
print(tf.__version__)

1.15.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
RESEARCH_DIR = "/content/drive/MyDrive/TFOD/models/models/research"

In [ ]:
os.chdir(RESEARCH_DIR)

In [ ]:
os.getcwd()

'/content/drive/MyDrive/TFOD/models/models/research'

In [ ]:
#!python xml_to_csv.py -i data_nameplate/images -o data_nameplate/annotations_test/train_label.csv -l data_nameplate/annotations

In [ ]:
#!python xml_to_csv.py -i data_nameplate/images_test -o data_nameplate/annotations_test/test_label.csv -l data_nameplate/annotations_test

In [ ]:
#!python generate_tfrecord.py --csv_input=data_nameplate/annotations_test/train_label.csv --output_path=data_nameplate/annotations/train.record --image_dir=data_nameplate/images --label_map training/labelmap.pbtxt

In [ ]:
#!python generate_tfrecord.py --csv_input=data_nameplate/annotations_test/test_label.csv --output_path=data_nameplate/annotations/test.record --image_dir=data_nameplate/images_test --label_map training/labelmap.pbtxt

In [ ]:
#!python eval.py --logtostderr --checkpoint_dir=training --eval_dir=/content/drive/MyDrive/TFOD/models/models/research/test_images_new --pipeline_config_path=/content/drive/MyDrive/TFOD/models/models/research/training/ssdlite_mobilenet_v2_coco.config

In [ ]:
!python model_main.py --logtostderr --model_dir=training/ --pipeline_config_path=training/ssdlite_mobilenet_v2_coco.config

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





W0322 09:21:00.857676 140038937687936 module_wrapper.py:139] From /content/drive/MyDrive/TFOD/models/models/research/object_detection/utils/config_util.py:94: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0322 09:21:01.075742 140038937687936 module_wrapper.py:139] From /content/drive/MyDrive/TFOD/models/models/research/object_detection/model_lib.py:573: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0322 09:21:01.075955 140038937687936 model_lib.py:574] Forced number of epochs for all eval validations to be 1.

W0322 09:21:01.076098 140038937687

In [ ]:
#!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync.config --trained_checkpoint_prefix training/model.ckpt-63261 --output_directory inference_graph_nameplate

In [ ]:
# !python export_tflite_ssd_graph.py \
#     --pipeline_config_path training/ssd_mobilenet_v1_coco.config \
#     --trained_checkpoint_prefix training/model.ckpt-114744 \
#     --output_directory inference_graph_tflite_nameplate

In [ ]:
# !tflite_convert --graph_def_file=inference_graph_tflite_nameplate/tflite_graph.pb --output_file=detect_nameplate.tflite \
# --input_shapes=1,300,300,3 --input_arrays=normalized_input_image_tensor \
# --output_arrays=TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3 --allow_custom_ops

In [ ]:
!pip install coremltools==4.0

In [ ]:
 #input_name_shape_dict={'input_1': (1, 300, 300, 3)},image_scale=1/255

In [ ]:
# import coremltools as ct
# model_input = ct.ImageType(shape=(1, 300, 300, 3))

In [ ]:
# model = ct.convert(
#     '/content/drive/MyDrive/TFOD/models/models/research/inference_graph/frozen_inference_graph.pb'
# ,inputs=[model_input])

In [ ]:
#!pip install tfcoreml

In [ ]:
print(os.getcwd())

/content/drive/MyDrive/TFOD/models/models/research


In [ ]:
#'detection_scores:0','detection_boxes:0','num_detections:0'

In [ ]:
model = "inference_graph/frozen_inference_graph.pb"
#model="inference_graph/saved_model"

In [ ]:
import tensorflow as tf

path = "inference_graph/frozen_inference_graph.pb"

# Load the protobuf file from the disk and parse it to retrieve the
# graph_def
with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

# Import the graph_def into a new Graph
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name="")

In [ ]:
ops = graph.get_operations()
N = len(ops)
# print all the placeholder ops, these would be the inputs
print("Inputs ops: ")
for op in ops:
    if op.type == "Placeholder":
        print("op name: {}, output shape : {}".
              format(op.name, op.outputs[0].get_shape()))

Inputs ops: 
op name: image_tensor, output shape : (?, ?, ?, 3)


In [ ]:
print("\nProspective output tensor(s): ", )
sink_ops = []
input_tensors = set()
for op in ops:
    for x in op.inputs:
        if x.name not in input_tensors:
            input_tensors.add(x.name)
for op in ops:
    if len(op.outputs) > 0:
        x = op.outputs[0]
        if x.name not in input_tensors:
            print("tensor name: {}, tensor shape : {}, parent op type: {}"
                  .format(x.name, x.get_shape(), op.type))


Prospective output tensor(s): 
tensor name: Preprocessor/map/while/Switch:0, tensor shape : <unknown>, parent op type: Switch
tensor name: Preprocessor/map/while/Switch_1:0, tensor shape : <unknown>, parent op type: Switch
tensor name: Postprocessor/BatchMultiClassNonMaxSuppression/map/while/Switch:0, tensor shape : <unknown>, parent op type: Switch
tensor name: Postprocessor/BatchMultiClassNonMaxSuppression/map/while/Switch_1:0, tensor shape : <unknown>, parent op type: Switch
tensor name: Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/SortByField/TopKV2:0, tensor shape : (?,), parent op type: TopKV2
tensor name: Postprocessor/BatchMultiClassNonMaxSuppression/map/while/MultiClassNonMaxSuppression/SortByField_1/TopKV2:0, tensor shape : (?,), parent op type: TopKV2
tensor name: detection_boxes:0, tensor shape : <unknown>, parent op type: Identity
tensor name: detection_scores:0, tensor shape : <unknown>, parent op type: Identity
tensor name: detect

In [ ]:
import coremltools
model = "inference_graph/frozen_inference_graph.pb"
outputs=['detection_classes:0','detection_scores:0','detection_boxes:0','num_detections:0']
coremltools.convert(model,outputs=outputs)

Converting Frontend ==> MIL Ops:  35%|███▍      | 249/719 [00:00<00:00, 1258.59 ops/s]

tf shape: [-1, -1, -1, 3]
inf shape: [is0, is1, is2, 3]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: [3, 3, 3, 32]
inf shape: [3, 3, 3, 32]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [3, 3, 32, 1]
inf shape: [3, 3, 32, 1]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [32]
inf shape: [32]
tf shape: [1, 1, 32, 64]
inf shape: [1, 1, 32, 64]
tf shape: [64]
inf shape: [64]
tf shape: [64]
inf shape: [64]
tf shape: [64]
inf shape: [64]
tf shape: 


Converting Frontend ==> MIL Ops: 100%|██████████| 9/9 [00:00<00:00, 4062.50 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 30/30 [00:00<00:00, 1357.47 ops/s]

Converting Frontend ==> MIL Ops: 100%|██████████| 9/9 [00:00<00:00, 2494.46 ops/s]

Converting Frontend ==> MIL Ops:  51%|█████     | 368/719 [00:00<00:00, 969.52 ops/s] 

tf shape: [1]
inf shape: [1]
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: [1, 1, 256, 24]
inf shape: [1, 1, 256, 24]
tf shape: [24]
inf shape: [24]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: [1, 1, 256, 18]
inf shape: [1, 1, 256, 18]
tf shape: [18]
inf shape: [18]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: [1, 1, 128, 24]
inf shape: [1, 1, 128, 24]
tf shape: [24]
inf shape: [24]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: []
inf shape: []
tf shape: [1, 1, 128, 18]
inf shape: [1, 1, 128, 18]
tf shape: [18]
inf shape: [18]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: [1]
inf shape: [1]
tf shape: []
inf 

Converting Frontend ==> MIL Ops:  60%|█████▉    | 429/719 [00:00<00:00, 692.40 ops/s]

...

Converting Frontend ==> MIL Ops:  84%|████████▍ | 606/719 [00:01<00:00, 239.49 ops/s]


inf shape: [4]
tf shape: [-1, -1, -1, 128]
inf shape: [is3, 3, 3, 128]
tf shape: [128]
inf shape: [128]
tf shape: [128]
inf shape: [128]
tf shape: [-1, -1, -1, 24]
inf shape: [is3, 3, 3, 24]
tf shape: []
inf shape: []
tf shape: [-1, -1, -1, 18]
inf shape: [is3, 3, 3, 18]
tf shape: []
inf shape: []
tf shape: [4]
inf shape: [4]
tf shape: [3]
inf shape: [3]
tf shape: [-1, -1, -1, 128]
inf shape: [is3, 3, 3, 128]
tf shape: [-1, -1, 1, 4]
inf shape: [is3, 54, 1, 4]
tf shape: [-1, -1, 3]
inf shape: [is3, 54, 3]
tf shape: [-1, -1, -1, 256]
inf shape: [is3, 2, 2, 256]
tf shape: [-1, -1, -1, 256]
inf shape: [is3, 2, 2, 256]
tf shape: [256]
inf shape: [256]
tf shape: [256]
inf shape: [256]
tf shape: [-1, -1, -1, 256]
inf shape: [is3, 2, 2, 256]
tf shape: [-1, -1, -1, 64]
inf shape: [is3, 2, 2, 64]
tf shape: [-1, -1, -1, 24]
inf shape: [is3, 2, 2, 24]
tf shape: [4]
inf shape: [4]
tf shape: [-1, -1, -1, 18]
inf shape: [is3, 2, 2, 18]
tf shape: [4]
inf shape: [4]
tf shape: [-1, -1, -1, 64]
inf sha

Converting Frontend ==> MIL Ops:  84%|████████▍ | 606/719 [00:01<00:00, 418.37 ops/s]



tf shape: [-1]
inf shape: [1917*is3]
tf shape: [3]
inf shape: [3]
tf shape: [-1, 1917, 4]
inf shape: [is34, is35, is36]


ValueError: ignored

In [ ]:
import coremltools
model=coremltools.converters.tensorflow.convert(
    model = "inference_graph/frozen_inference_graph.pb",
    mlmodel_path = "inference_graph/optimized_graph.mlmodel",
    output_feature_names =['detection_classes:0','detection_scores:0','detection_boxes:0','num_detections:0'],
    input_name_shape_dict={"image_tensor:0":[1,300,300,3]}
)

In [ ]:
import tfcoreml as tf_converter
tf_converter.convert(
    tf_model_path = "/content/drive/MyDrive/TFOD/models/models/research/inference_graph/frozen_inference_graph.pb",
    mlmodel_path = "inference_graph/optimized_graph.mlmodel",
    output_feature_names=['detection_classes:0','detection_scores:1','detection_boxes:2','num_detections:3'],
    input_name_shape_dict={"image_tensor:0":[1,300,300,3]},
    minimum_ios_deployment_target='12'
)

In [ ]:
tfcoreml.convert(tf_model_path='my_model.pb',
                 mlmodel_path='my_model.mlmodel',
                 output_feature_names=['softmax:0'],  # name of the output tensor (appended by ":0")
                 input_name_shape_dict={'input:0': [1, 227, 227, 3]},  # map from input tensor name (placeholder op in the graph) to shape
                 minimum_ios_deployment_target='12')

In [ ]:
#  image_input_names = 'Mul:0',
#  class_labels = 'labels.txt'
#  image_input_names = 'Mul:0',
#  class_labels = 'labels.txt'